# Sentiment Analysis dari Instagram comments calon Presiden 2024 sebelum dan sesudah deklrasi calon wakil presiden menggunakan Naive Bayes


## Tujuan : mengetahui perbedaan sentiment dari komentar instagram sebelum dan sesudah deklarasi calon wakil presiden


<ul>
    <li>
    Aurelius Ivan Wijaya (00000054769)
    </li>
    <li>
    Rajendra Abhinaya (00000060445)
    </li>
    <li>
    Maecyntha Irelynn Tantra (00000055038)
    </li>
    <li>
    Patricia theodora (00000054093)
    </li>
<ul>

# Data Collection

apa sih yang sebenernya kita cari?
* web scrapping algoritm untuk data primer (ivan)
* labeling (pat)
* data sekunder (mae)
* stopword library indonesia (abhi)
* cari jurnal referensi yang sudah, sebagai literature review (all, min 4 per person)

In [302]:
import pandas as pd
import numpy as np

In [303]:
# import dataset    
# [ Primary Dataset ]
# anies_before = pd.read_csv('./Dataset/Anies/anies_before.csv', on_bad_lines='skip')
# anies_after = pd.DataFrame(pd.read_csv('./Dataset/Anies/anies_after.csv', on_bad_lines="skip"))
ganjar_before = pd.DataFrame(pd.read_csv('./Dataset/Ganjar/ganjar_before.csv', on_bad_lines="skip")) # get only 100 data
ganjar_before = ganjar_before[:100]
ganjar_after = pd.DataFrame(pd.read_csv('./Dataset/Ganjar/ganjar_after.csv', on_bad_lines="skip"))
ganjar_after = ganjar_after[:100]
prabowo_before = pd.DataFrame(pd.read_csv('./Dataset/Prabowo/prabowo_before.csv', on_bad_lines="skip"))
prabowo_before = prabowo_before[:100]
prabowo_after = pd.DataFrame(pd.read_csv('./Dataset/Prabowo/prabowo_after.csv', on_bad_lines="skip"))
prabowo_after = prabowo_after[:100]
# prabowo_before = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_prabowo_before.csv', on_bad_lines="skip"))
# prabowo_after = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_prabowo_after.csv', on_bad_lines="skip"   ))
detik_anies_before = pd.read_csv('./Dataset/Detik/detik_anies_before.csv', on_bad_lines='skip')
detik_anies_after = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_anies_before.csv', on_bad_lines="skip"))
detik_ganjar_before = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_ganjar_before.csv', on_bad_lines="skip"))
detik_ganjar_after = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_ganjar_after.csv', on_bad_lines="skip"))
detik_prabowo_before = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_prabowo_before.csv', on_bad_lines="skip"))
detik_prabowo_after = pd.DataFrame(pd.read_csv('./Dataset/Detik/detik_prabowo_after.csv', on_bad_lines="skip"   ))
# [ Secondary Dataset ]
instagram_cyber_comments = pd.read_csv('./Dataset/dataset_komentar_instagram_cyberbullying.csv')
tweet_tv = pd.read_csv('./Dataset/dataset_tweet_sentimen_tayangan_tv.csv')
tweet_pilkada = pd.read_csv('./Dataset/dataset_tweet_sentiment_pilkada_DKI_2017.csv')
tweet_opini_film = pd.read_csv('./Dataset/dataset_tweet_sentiment_opini_film.csv')
tweet_cellular = pd.read_csv('./Dataset/dataset_tweet_sentiment_cellular_service_provider.csv')
prastyo_sentiment = pd.read_csv('./Dataset/prastyo-sentiment_all.csv')
sentiment_twitter_indonesia = pd.read_csv('./Dataset/Indonesian Sentiment Twitter Dataset Labeled.csv', sep='\t')
# sentiment_shopee = pd.read_csv('./Dataset/dataset_shopee2.csv', on_bad_lines="skip")
import chardet

# Detect encoding
with open('./Dataset/dataset_shopee2.csv', 'rb') as f:
    result = chardet.detect(f.read())
# Use the detected encoding when reading the CSV file
sentiment_shopee = pd.read_csv('./Dataset/dataset_shopee2.csv', on_bad_lines="skip", encoding=result['encoding'])

In [304]:
import pandas as pd

def balance_dataset(data, label_column, max_samples=None):

    # Determine the minimum number of samples for any label
    min_samples = data[label_column].value_counts().min()

    # Randomly select min_samples samples for each label
    balanced_subset = pd.concat([group.sample(min_samples) for _, group in data.groupby(label_column)])

    # Shuffle the balanced subset to mix the labels
    balanced_subset = balanced_subset.sample(frac=1).reset_index(drop=True)

    # Optionally only keep a maximum number of samples
    if max_samples is not None:
        balanced_subset = balanced_subset[:max_samples]
    
    return balanced_subset

## Data Integration

In [305]:
# 1: positive, 0: negative 

# data integration
# change label name to 'comments'
instagram_cyber_comments.rename(columns={'Instagram Comment Text': 'comments'}, inplace=True)
# mapping sentiment
instagram_cyber_comments['Sentiment'] = instagram_cyber_comments['Sentiment'].map({'positive': 1, 'negative': 0})
instagram_cyber_comments['label'] = instagram_cyber_comments['Sentiment'].astype(int)
# drop unused columns
instagram_cyber_comments.drop(columns=['Id', 'Sentiment'], inplace=True)
# display(instagram_cyber_comments.head())
#change data type to string to ensure all data type is string
instagram_cyber_comments['comments'] = instagram_cyber_comments['comments'].astype(str)

# tweet_tv
# change label name to 'comments'
tweet_tv.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_tv['Sentiment'] = tweet_tv['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_tv['label'] = tweet_tv['Sentiment'].astype(int)
tweet_tv.rename(columns={'Text Tweet': 'comments'}, inplace=True)
tweet_tv.drop(columns=['Id', 'Sentiment', "Jumlah Retweet", "Acara TV"], inplace=True)
#change data type to string to ensure all data type is string
tweet_tv['comments'] = tweet_tv['comments'].astype(str)

# tweet_pilkada
# tweet_pilkada add new column 'label'
# tweet_pilkada['label'] = tweet_pilkada['Sentiment'].map({'positive': 1, 'negative': 0}).astype(int)
tweet_pilkada['Sentiment'] = tweet_pilkada['Sentiment'].map({'positive': 1, 'negative': 0}).astype(int)
# tweet_pilkada['label'] = tweet_pilkada['Sentiment'].astype(int)
tweet_pilkada = tweet_pilkada[['Sentiment', 'Text Tweet']]
tweet_pilkada.rename(columns={'Text Tweet': 'comments'}, inplace=True)
tweet_pilkada.rename(columns={'Sentiment': 'label'}, inplace=True)
#change data type to string to ensure all data type is string
tweet_pilkada['comments'] = tweet_pilkada['comments'].astype(str)

# tweet_opini_film
# change label name to 'comments'
tweet_opini_film.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_opini_film['Sentiment'] = tweet_opini_film['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_opini_film['label'] = tweet_opini_film['Sentiment'].astype(int)
tweet_opini_film.drop(columns=['Id', 'Sentiment'], inplace=True)
tweet_opini_film.rename(columns={'Text Tweet': 'comments'}, inplace=True)
#change data type to string to ensure all data type is string
tweet_opini_film['comments'] = tweet_opini_film['comments'].astype(str)

# tweet_cellular
# change label name to 'comments'
tweet_cellular.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_cellular['Sentiment'] = tweet_cellular['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_cellular['label'] = tweet_cellular['Sentiment'].astype(int)
tweet_cellular.drop(columns=['Id', 'Sentiment'], inplace=True)
tweet_cellular.rename(columns={'Text Tweet': 'comments'}, inplace=True)

# prastyo_sentiment
map = {'pos': 1, 'neg': 0, 'neu': 1}
prastyo_sentiment['label'] = prastyo_sentiment['label'].map(map)


# sentiment twitter indonesia
# change label name to 'comments'
sentiment_twitter_indonesia.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
# sentiment_twitter_indonesia['sentimen'] = sentiment_twitter_indonesia['sentimen'].map({'positive': 1, 'negative': 0})
sentiment_twitter_indonesia['label'] = sentiment_twitter_indonesia['sentimen'].astype(int)


# shoppe
# change label name to 'comments'
sentiment_shopee.rename(columns={'Review': 'comments'}, inplace=True)
# mapping sentiment
sentiment_shopee['SENTIMEN'] = sentiment_shopee['SENTIMEN'].map({'POSITIF': 1, 'NEGATIF': 0})
sentiment_shopee['label'] = sentiment_shopee['SENTIMEN'].astype(int)
sentiment_shopee.drop(columns=['SENTIMEN'], inplace=True)


# detik_anies_after = balance_dataset(detik_anies_after, 'label')
# detik_anies_before = balance_dataset(detik_anies_before, 'label')
# detik_ganjar_before = balance_dataset(detik_ganjar_before, 'label', max_samples=50)
detik_ganjar_after = balance_dataset(detik_ganjar_after, 'label', max_samples=50)
tweet_pilkada = balance_dataset(tweet_pilkada, 'label')

Training_dataset = pd.read_csv('./Dataset/Training/Training.csv', on_bad_lines='skip')
# only pick comments and label column
Training_dataset = Training_dataset[['comments', 'label']]
secondary_dataset = pd.concat([
   Training_dataset,
    ], ignore_index=True)
# only pick comments and label column
secondary_dataset = secondary_dataset[['comments', 'label']]
#change data type to string to ensure all data type is string
secondary_dataset['comments'] = secondary_dataset['comments'].astype(str)
# if there is column 'Unnamed: 0' in secondary dataset, drop it
if 'Unnamed: 0' in secondary_dataset.columns:
    secondary_dataset.drop(columns=['Unnamed: 0'], inplace=True)

# Pre-Proccessing

## Data Cleaning

### Handle Missing Value

In [306]:
display(secondary_dataset.info())
display(secondary_dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  111 non-null    object
 1   label     111 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


None

,comments,label
0,Kalo saingannya pak prabowo dan pak eric ya su...,0
1,Gas❤ saya pilih pak ganjar,1
2,"Prestasi enggak ada, suka nonton film porno pu...",0
3,Pak Anies Baswedan Presiden RI 2024😍 Kecerdasa...,1
4,"Siapapun wapresnya, Insya Allah saya akan meya...",1


In [307]:
def handleMissingValue(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df[df['comments'] != '']
    # remove 'NaN' value
    if('NaN' in df['comments']):
        df = df[df['comments'] != 'NaN']
    df = df[df['comments'].notna()]
    df = df.reset_index(drop=True)
    df = df[['comments']] 
    return df
secondary_dataset['comments'] = handleMissingValue(secondary_dataset)
display(secondary_dataset.info())
display(secondary_dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  111 non-null    object
 1   label     111 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


None

,comments,label
0,Kalo saingannya pak prabowo dan pak eric ya su...,0
1,Gas❤ saya pilih pak ganjar,1
2,"Prestasi enggak ada, suka nonton film porno pu...",0
3,Pak Anies Baswedan Presiden RI 2024😍 Kecerdasa...,1
4,"Siapapun wapresnya, Insya Allah saya akan meya...",1


In [308]:
print(secondary_dataset.isnull().sum())
# delete duplicate data and missing value and null value
secondary_dataset = secondary_dataset.dropna()
secondary_dataset = secondary_dataset.drop_duplicates()
secondary_dataset = secondary_dataset[secondary_dataset['comments'] != '']
# remove 'NaN' value
if('NaN' in secondary_dataset['comments']):
    secondary_dataset = secondary_dataset[secondary_dataset['comments'] != 'NaN']
secondary_dataset = secondary_dataset[secondary_dataset['comments'].notna()]
secondary_dataset = secondary_dataset.reset_index(drop=True)
# secondary_dataset = secondary_dataset[['comments']]
print(secondary_dataset.isnull().sum())

display(secondary_dataset)

comments    0
label       0
dtype: int64
comments    0
label       0
dtype: int64


,comments,label
0,Kalo saingannya pak prabowo dan pak eric ya su...,0
1,Gas❤ saya pilih pak ganjar,1
2,"Prestasi enggak ada, suka nonton film porno pu...",0
3,Pak Anies Baswedan Presiden RI 2024😍 Kecerdasa...,1
4,"Siapapun wapresnya, Insya Allah saya akan meya...",1
...,...,...
106,Ngebacot mah...tukang becak juga bisa ngomong ...,0
107,yah kecewa. Harusnya Pak Erick Tohir. Pasti ta...,0
108,Manusia ambisius berkuasa sampai MK jadi alat ...,0
109,"blunder pak jokowi, kenapa enggak erick atau m...",0


### Text Cleaning

In [309]:
import string
import re
import pandas as pd

# Precompile regex patterns
re_long_word = re.compile(r'\b\w{20,}\b')
re_non_ascii = re.compile(r'[^\x00-\x7F]+')
# re_punctuation = re.compile(f"[{re.escape(string.punctuation)}]")
re_single_char = re.compile(r'\s+[a-zA-Z]\s+')
re_numbers = re.compile(r'\d+')
re_tags = re.compile("&lt;/?.*?&gt;")
re_special_chars_digits = re.compile("(\\d|\\W)+")
re_repeated_chars = re.compile(r'(.)\1+')

import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def clean_text(comment):
    comment = str(comment)
    comment = comment.lower() # Case Folding to lowercase

    # Remove non-ascii words and characters
    comment = re_non_ascii.sub(' ', comment) # remove non-ascii characters

    # Remove all links
    comment = re.sub(r"http\S+", "", comment) # remove all links

    # Delete all emoticons from text
    comment = re.sub(':[^ ]+:', '', comment) # remove emoticons
    comment = remove_emojis(comment)
    # remove all word that start with < and end with >
    comment = re.sub('<[^>]*>', '', comment)
    # Remove long words
    comment = re_long_word.sub('', comment) # remove long words

    # if length of comment is more than 7 and ended with "ny" convert it to "nya"
    if(len(comment) > 7 and comment[-2:] == 'ny'):
        # comment = comment[:-2] + 'nya'
        # add nya to the end of word
        comment = re.sub(r'\b(\w+)', r'\1nya', comment)
    # Remove single characters
    comment = re_single_char.sub(' ', comment) # remove single characters

    # Remove numbers
    comment = re_numbers.sub('', comment)

    # Remove word that contains tags
    comment = re_tags.sub('', comment)

    # Remove special characters and digits
    comment = re_special_chars_digits.sub(" ", comment) 

    # If a letter is repeated more than 2 times, replace it with 1 time
    comment = re_repeated_chars.sub(r'\1\1', comment)
    # if comment is empty or only contain one word, return empty string
    if(len(comment) <= 1):
        return ''
    # Remove extra whitespaces
    comment = re.sub(r'\s+', ' ', comment)

    return comment.strip()

def remove_punct(data):
    data['comments'] = data['comments'].apply(clean_text)
    # remove nan value
    data = data[data['comments'] != '']
    data = data[data['comments'].notna()]
    data = data.reset_index(drop=True)
    return data

In [310]:
# Call the function
secondary_dataset.to_csv('./dirty_data.csv', index=False)
cleaned_data = remove_punct(secondary_dataset)
secondary_dataset['comments'] = cleaned_data['comments']

secondary_dataset = secondary_dataset[secondary_dataset['comments'] != '']
# remove 'NaN' value
if('NaN' in secondary_dataset['comments']):
    secondary_dataset = secondary_dataset[secondary_dataset['comments'] != 'NaN']
secondary_dataset = secondary_dataset[secondary_dataset['comments'].notna()]
secondary_dataset = secondary_dataset.reset_index(drop=True)

# secondary_dataset = secondary_dataset[['comments']]
print(secondary_dataset.isnull().sum())
# Save to CSV

comments    0
label       0
dtype: int64


In [311]:
display(secondary_dataset)

,comments,label
0,kalo saingannya pak prabowo dan pak eric ya su...,0
1,gas saya pilih pak ganjar,1
2,prestasi enggak ada suka nonton film porno pul...,0
3,pak anies baswedan presiden ri kecerdasan cint...,1
4,siapapun wapresnya insya allah saya akan meyak...,1
...,...,...
106,ngebacot mah tukang becak juga bisa ngomong do...,0
107,yah kecewa harusnya pak erick tohir pasti tamb...,0
108,manusia ambisius berkuasa sampai mk jadi alat ...,0
109,blunder pak jokowi kenapa enggak erick atau ma...,0


### Stopword Removal

In [312]:
import re

display(secondary_dataset)
# Stopword removal

stopword_bahasa = pd.read_csv('./Dataset/TextNormalization/stopwordbahasa.csv', header=None)

# def stopword_removal(data):
#     for i in range(len(data)):
#         data = data.replace(stopword_bahasa[0][i], '')
#     return data

def stopword_removal(data):
    if isinstance(data, str):
        # Split the string into a list of words
        words = data.split()
        # Remove stop words
        words = [word for word in words if word not in stopword_bahasa[0].values]
        # Join the list of words back into a string
        return ' '.join(words)
    return data


,comments,label
0,kalo saingannya pak prabowo dan pak eric ya su...,0
1,gas saya pilih pak ganjar,1
2,prestasi enggak ada suka nonton film porno pul...,0
3,pak anies baswedan presiden ri kecerdasan cint...,1
4,siapapun wapresnya insya allah saya akan meyak...,1
...,...,...
106,ngebacot mah tukang becak juga bisa ngomong do...,0
107,yah kecewa harusnya pak erick tohir pasti tamb...,0
108,manusia ambisius berkuasa sampai mk jadi alat ...,0
109,blunder pak jokowi kenapa enggak erick atau ma...,0


In [313]:

# Call the function
# cleaned_data = stopword_removal(secondary_dataset)
# convert comments to string
# secondary_dataset['comments'] = secondary_dataset['comments'].astype(str)

#check null value
# secondary_dataset['comments'] = secondary_dataset['comments'].apply(stopword_removal)
secondary_dataset['comments'] = secondary_dataset['comments'].apply(lambda x: stopword_removal(x))
# secondary_dataset['comments'] = secondary_dataset['comments'].apply(lambda x: stopword_removal(x))
display(secondary_dataset)

# handle null, missing value, and duplicate data
secondary_dataset = secondary_dataset.dropna()
secondary_dataset = secondary_dataset.drop_duplicates()
secondary_dataset = secondary_dataset[secondary_dataset['comments'] != '']
# remove 'NaN' value
if('NaN' in secondary_dataset['comments']):
    secondary_dataset = secondary_dataset[secondary_dataset['comments'] != 'NaN']
secondary_dataset = secondary_dataset[secondary_dataset['comments'].notna()]
secondary_dataset = secondary_dataset.reset_index(drop=True)
# secondary_dataset = secondary_dataset[['comments']]
display(secondary_dataset)

,comments,label
0,kalo saingannya ya melenggang,0
1,gas pilih,1
2,prestasi suka nonton film porno pulak gini cal...,0
3,presiden ri kecerdasan cinta rakyat udh no,1
4,wapresnya insya allah orang memenangkan sbg pr...,1
...,...,...
106,ngebacot mah tukang becak ngomong doang mah,0
107,yah kecewa erick tohir kesempatan menang dibandin,0
108,manusia ambisius berkuasa mk alat politik kelu...,0
109,blunder erick aduh kalah aja inimah,0


,comments,label
0,kalo saingannya ya melenggang,0
1,gas pilih,1
2,prestasi suka nonton film porno pulak gini cal...,0
3,presiden ri kecerdasan cinta rakyat udh no,1
4,wapresnya insya allah orang memenangkan sbg pr...,1
...,...,...
105,ngebacot mah tukang becak ngomong doang mah,0
106,yah kecewa erick tohir kesempatan menang dibandin,0
107,manusia ambisius berkuasa mk alat politik kelu...,0
108,blunder erick aduh kalah aja inimah,0


### Tokenization

In [314]:
# Tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
secondary_dataset['comments'] = secondary_dataset['comments'].apply(word_tokenize)

display(secondary_dataset)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aurel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,comments,label
0,"[kalo, saingannya, ya, melenggang]",0
1,"[gas, pilih]",1
2,"[prestasi, suka, nonton, film, porno, pulak, g...",0
3,"[presiden, ri, kecerdasan, cinta, rakyat, udh,...",1
4,"[wapresnya, insya, allah, orang, memenangkan, ...",1
...,...,...
105,"[ngebacot, mah, tukang, becak, ngomong, doang,...",0
106,"[yah, kecewa, erick, tohir, kesempatan, menang...",0
107,"[manusia, ambisius, berkuasa, mk, alat, politi...",0
108,"[blunder, erick, aduh, kalah, aja, inimah]",0


### Text Normalization

In [315]:
# Text Normalization / Noise Removal
indo_slang_word = pd.read_csv("./Dataset/TextNormalization/lexicontest.csv")

display(indo_slang_word.shape)
display(indo_slang_word.head())

(5045, 2)

,slang,formal
0,22nya,dua-duanya
1,aaaaaaakkk,ah
2,aaaaahhhh,ah
3,aaaammmiiiinnnn,amin
4,aaahh,ah


In [316]:
# # def replace_slang_word(doc,slang_word):
# #     for index in  range(0,len(doc)-1):
# #         index_slang = slang_word.slang==doc[index]
# #         formal = list(set(slang_word[index_slang].formal))
# #         # replace slang word with formal word
# #         if len(formal)>0:
# #             doc[index] = formal[0]
        
# #     return doc
# def replace_slang_word(doc, slang_word):
#     for index in range(len(doc)):
#         slang_match = slang_word[slang_word['slang'].isin([doc[index]])]
#         if not slang_match.empty:
#             formal = list(set(slang_match['formal']))
#             if len(formal) > 0:
#                 doc[index] = formal[0]
#     return doc

# prevent word to cut in the middle of word for example: 'kemarin' -> 'kemarin' not 'kemar' 'in'

def replace_slang_word(doc, slang_word):
    for index in range(len(doc)):
        slang_match = slang_word[slang_word['slang'].isin([doc[index]])]
        if not slang_match.empty:
            formal = list(set(slang_match['formal']))
            if len(formal) > 0:
                doc[index] = formal[0]
    return doc

In [317]:
print(secondary_dataset)
# convert data frame to array
# secondary_dataset['comments'] = secondary_dataset['comments'].apply(lambda x: np.array(x))
normal_data = secondary_dataset
normal_data = secondary_dataset['comments'].apply(replace_slang_word, slang_word=indo_slang_word)
# append data to secondary_dataset
print(secondary_dataset)

#testing
testing = secondary_dataset['comments'][0]
print(testing)
testing = replace_slang_word(testing, indo_slang_word)

                                              comments  label
0                   [kalo, saingannya, ya, melenggang]      0
1                                         [gas, pilih]      1
2    [prestasi, suka, nonton, film, porno, pulak, g...      0
3    [presiden, ri, kecerdasan, cinta, rakyat, udh,...      1
4    [wapresnya, insya, allah, orang, memenangkan, ...      1
..                                                 ...    ...
105  [ngebacot, mah, tukang, becak, ngomong, doang,...      0
106  [yah, kecewa, erick, tohir, kesempatan, menang...      0
107  [manusia, ambisius, berkuasa, mk, alat, politi...      0
108         [blunder, erick, aduh, kalah, aja, inimah]      0
109  [ogah, milih, lu, petugas, partai, menggagalka...      0

[110 rows x 2 columns]
                                              comments  label
0                  [kalo , saingannya, ya, melenggang]      0
1                                         [ayo, pilih]      1
2    [prestasi, suka, nonton, film, porno, pul

### Stemming

In [318]:
# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [319]:
def lemma_indo(doc):
    return [stemmer.stem(word) for word in doc]

In [320]:
# stemming
display(secondary_dataset['comments'][0])
stemmed_data = secondary_dataset['comments'].apply(lemma_indo)
# append stemmed data to secondary dataset, don't forget to reset index
secondary_dataset['comments'] = stemmed_data
secondary_dataset = secondary_dataset.reset_index(drop=True)
display(secondary_dataset)

['kalo ', 'saingannya', 'ya', 'melenggang']

,comments,label
0,"[kalo, saing, ya, lenggang]",0
1,"[ayo, pilih]",1
2,"[prestasi, suka, nonton, film, porno, pula, be...",0
3,"[presiden, ri, cerdas, cinta, rakyat, sudah, no]",1
4,"[wapresnya, insya, allah, orang, menang, bagai...",1
...,...,...
105,"[ngebacot, mah, tukang, becak, omong, saja, mah]",0
106,"[ya, kecewa, erick, tohir, sempat, menang, dib...",0
107,"[manusia, ambisius, kuasa, maka, alat, politik...",0
108,"[blunder, erick, aduh, kalah, saja, inimah]",0


## Synthetic Minority Oversampling Technique (SMOTE)

* this one need research (ivan)
* Smote adalah sebuah tehnik yang digunakan terhadap data yang tidak seimbang

In [321]:
# Smote
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# clean empty row
# Instantiate SMOTE
smote = SMOTE(
    sampling_strategy='minority',
    random_state=42
)
# Assuming 'comments' is the name of the text column
comments = secondary_dataset['comments']
# Assuming 'label' is name of the target column
y = secondary_dataset['label']


# vectorizer tokenized comments
# vectorizer = CountVectorizer()
# TF-IDF vectorization
vectorizer = TfidfVectorizer()

# vectorizer = CountVectorizer()
# X_tfidf = vectorizer.fit_transform(comments)
joined_comments = pd.DataFrame(comments.apply(lambda x: ' '.join(x)))
# display(joined_comments)
# X_tfidf = vectorizer.fit_transform(joined_comments) # if we want to vectorize all comments in one row
# fit vectorizer
vectorizer.fit(joined_comments['comments'])
# transform comments
X_tfidf = vectorizer.transform(joined_comments['comments'])
# vectorizer.fit(comments)
# X_tfidf = vectorizer.transform(comments)
# # display(X_tfidf)
# # display(X_tfidf.shape)
# Apply SMOTE to data
X_smote, y_smote = smote.fit_resample(X_tfidf, y)

In [322]:
# without SMOTE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
# random forest
from sklearn.ensemble import RandomForestClassifier
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# SVM
from sklearn.svm import SVC

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Instantiate Multinomial Naive Bayes classifier
classifier = MultinomialNB()
# Logistic Regression
classifier = LogisticRegression(random_state=42)
# Fit the classifier on the training data
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

Accuracy: 0.59
Classification Report:
               precision    recall  f1-score   support

           0       0.59      1.00      0.74        13
           1       0.00      0.00      0.00         9

    accuracy                           0.59        22
   macro avg       0.30      0.50      0.37        22
weighted avg       0.35      0.59      0.44        22



c:\Users\Aurel\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aurel\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aurel\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [323]:
# with SMOTE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
# random forest
from sklearn.ensemble import RandomForestClassifier
# Logistic Regression
from sklearn.linear_model import LogisticRegression

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Instantiate Multinomial Naive Bayes classifier
classifier = MultinomialNB(alpha=0.1)

# Fit the classifier on the training data
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)


Accuracy: 0.76
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.53      0.70        15
           1       0.67      1.00      0.80        14

    accuracy                           0.76        29
   macro avg       0.83      0.77      0.75        29
weighted avg       0.84      0.76      0.75        29



## Grid Search for Hyperparameter Tuning

In [324]:
from sklearn.model_selection import GridSearchCV
# import kfold
from sklearn.model_selection import KFold
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# All naive bayes
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# Create k-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Hyperparameter tuning for combination of alpha and fit_prior
model_param_grid = {
    'alpha': [0.01, 0.05,0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 100.0],
    'fit_prior': [True, False],
    'class_prior': [None, [0.25, 0.75], [0.5, 0.5], [0.75, 0.25]]
    }
# model_param_grid = {
#     'alpha': [0.01, 0.05,0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 100.0],
#     'fit_prior': [True, False],
#     }
# for gaussian naive bayes

model_grid_search = GridSearchCV(MultinomialNB(), model_param_grid, cv=kfold)
model_grid_search.fit(X_smote, y_smote)
# Print the best parameters and corresponding accuracy
print("\nBest parameters for MultinomialNB:", model_grid_search.best_params_)
print("Best accuracy for MultinomialNB:", model_grid_search.best_score_)

best_alpha = model_grid_search.best_params_['alpha']
best_fit_prior = model_grid_search.best_estimator_.fit_prior

display(best_alpha)
display(best_fit_prior)


Best parameters for MultinomialNB: {'alpha': 0.05, 'class_prior': [0.75, 0.25], 'fit_prior': True}
Best accuracy for MultinomialNB: 0.9012315270935961


0.05

True

## K-Fold Cross Validation

* this one need research (abhi)

In [325]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
# all forest algorithm
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier

# # Define 10-fold cross-validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# # Instantiate classifier 
classifier = MultinomialNB(alpha=best_alpha, fit_prior=best_fit_prior)

cv_results = cross_val_score(classifier, X_smote, y_smote, cv=kfold)
# # Print the results for each fold
best_fold = 0
best_accuracy = 0

for i, accuracy in enumerate(cv_results):
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_fold = i
        print(f"Fold {i + 1}: Accuracy = {accuracy} <== BEST RESULT")
    else:
        print(f"Fold {i + 1}: Accuracy = {accuracy}")

# from best fold, we will get the best model
# Instantiate classifier
# Print the mean and standard deviation of the cross-validation results
print(f"\nMean Accuracy: {cv_results.mean()}")
print(f"Standard Deviation: {cv_results.std()}")
# # After k-folds, now we will use the best model to predict the data

Fold 1: Accuracy = 0.6 <== BEST RESULT
Fold 2: Accuracy = 0.8 <== BEST RESULT
Fold 3: Accuracy = 0.7857142857142857
Fold 4: Accuracy = 0.8571428571428571 <== BEST RESULT
Fold 5: Accuracy = 0.7857142857142857
Fold 6: Accuracy = 0.9285714285714286 <== BEST RESULT
Fold 7: Accuracy = 0.7142857142857143
Fold 8: Accuracy = 1.0 <== BEST RESULT
Fold 9: Accuracy = 0.8571428571428571
Fold 10: Accuracy = 0.8571428571428571

Mean Accuracy: 0.8185714285714285
Standard Deviation: 0.10518205790971409


## Workload Distribution?

# Modeling

<!-- * kemungkinan Binomial Naive Bayes -->

In [326]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

# Load the dataset

classifier.fit(X_smote, y_smote)

def prediction(data):    
    tokenized = []
    for i in range(0, len(data)):
        # Check if the value is NaN or not a string
        if pd.notna(data[i]) and isinstance(data[i], str):
            tokenized.extend(word_tokenize(data[i]))
    # Transform using the same vectorizer used during training
    text = vectorizer.transform([" ".join(tokenized)]).toarray()
    # return classifier.predict(text)
    return classifier.predict(text)

In [327]:
import pandas as pd

def process_and_save_data(X, prediction_function, output_path='./temp.csv'):
    pos = 0
    neg = 0
    label = []
    predictionLabel = []

    for i in range(0, len(X)):
        text = [X['comments'][i]]
        pred = prediction_function(text)
        predictionLabel.append(pred[0])
        if pred == 1:
            pos += 1
        else:
            neg += 1
        label.append(pred[0])

    X['label'] = label

    # Save the DataFrame to a CSV file
    X.to_csv(output_path, index=False)
    print("Positive:", pos)
    print("Negative:", neg)
    # print percentage of positive and negative
    print("Positive percentage:", pos/len(X)*100)
    print("Negative percentage:", neg/len(X)*100)
    return pos, neg, predictionLabel

In [328]:
# Assuming 'comments' is the text data in your dataset
# X = pd.read_csv('./Dataset/Detik/', on_bad_lines='skip')
X = detik_anies_before
# clean data
X = X.dropna()
X = remove_punct(X)
# norm
X['comments'] = X['comments'].apply(word_tokenize)
X['comments'] = X['comments'].apply(replace_slang_word, slang_word=indo_slang_word)
X['comments'] = X['comments'].apply(lemma_indo)
# remove stopword

# convert back to string
X['comments'] = X['comments'].apply(lambda x: ' '.join(x))
# X['comments'] = X['comments'].apply(stopword_removal)
print(X['label'].value_counts())
X['comments'] = X['comments'].astype(str)
true_label = X['label']
pos, neg, predictionLabel = process_and_save_data(X, prediction, './temp.csv')

print("\n")
from sklearn.metrics import classification_report
print(classification_report(true_label, predictionLabel))

# accuracy 75%
# accuracy 76%
# accuracy 74%

C:\Users\Aurel\AppData\Local\Temp\ipykernel_24520\3654328601.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comments'] = data['comments'].apply(clean_text)


label
0.0    305
1.0     48
Name: count, dtype: int64
Positive: 86
Negative: 267
Positive percentage: 24.362606232294617
Negative percentage: 75.63739376770539


              precision    recall  f1-score   support

         0.0       0.90      0.79      0.84       305
         1.0       0.24      0.44      0.31        48

    accuracy                           0.74       353
   macro avg       0.57      0.61      0.58       353
weighted avg       0.81      0.74      0.77       353



In [329]:
# Assuming 'comments' is the text data in your dataset
# X = pd.read_csv('./Dataset/Detik/', on_bad_lines='skip')
X = detik_ganjar_before
# see how much true label
print(X['label'].value_counts())
# clean data
X = X.dropna()
X = remove_punct(X)
# norm
X['comments'] = X['comments'].apply(word_tokenize)
X['comments'] = X['comments'].apply(replace_slang_word, slang_word=indo_slang_word)
X['comments'] = X['comments'].apply(lemma_indo)
# remove stopword

# convert back to string
X['comments'] = X['comments'].apply(lambda x: ' '.join(x))
# X['comments'] = X['comments'].apply(stopword_removal)
print(X['label'].value_counts())
X['comments'] = X['comments'].astype(str)
true_label = X['label']
pos, neg, predictionLabel = process_and_save_data(X, prediction, './temp.csv')

print("\n")
from sklearn.metrics import classification_report
print(classification_report(true_label, predictionLabel))

# accuracy 87%
# accuracy 90%
# accuracy 89%

label
0    72
1    44
Name: count, dtype: int64
label
0    72
1    44
Name: count, dtype: int64
Positive: 43
Negative: 73
Positive percentage: 37.06896551724138
Negative percentage: 62.93103448275862


              precision    recall  f1-score   support

           0       0.90      0.92      0.91        72
           1       0.86      0.84      0.85        44

    accuracy                           0.89       116
   macro avg       0.88      0.88      0.88       116
weighted avg       0.89      0.89      0.89       116



In [330]:
X = Training_dataset
# see how much true label
print(X['label'].value_counts())
# clean data
X = X.dropna()
X = remove_punct(X)
# norm
X['comments'] = X['comments'].apply(word_tokenize)
X['comments'] = X['comments'].apply(replace_slang_word, slang_word=indo_slang_word)
X['comments'] = X['comments'].apply(lemma_indo)
# remove stopword

# convert back to string
X['comments'] = X['comments'].apply(lambda x: ' '.join(x))
# X['comments'] = X['comments'].apply(stopword_removal)
print(X['label'].value_counts())
X['comments'] = X['comments'].astype(str)
true_label = X['label']
pos, neg, predictionLabel = process_and_save_data(X, prediction, './temp.csv')

print("\n")
from sklearn.metrics import classification_report
print(classification_report(true_label, predictionLabel))

# accuracy 90%
# accuracy 94%

label
0    72
1    39
Name: count, dtype: int64
label
0    72
1    39
Name: count, dtype: int64
Positive: 38
Negative: 73
Positive percentage: 34.234234234234236
Negative percentage: 65.76576576576578


              precision    recall  f1-score   support

           0       0.96      0.97      0.97        72
           1       0.95      0.92      0.94        39

    accuracy                           0.95       111
   macro avg       0.95      0.95      0.95       111
weighted avg       0.95      0.95      0.95       111



In [331]:
X = detik_anies_after
# see how much true label
print(X['label'].value_counts())
# clean data
X = X.dropna()
X = remove_punct(X)
# norm
X['comments'] = X['comments'].apply(word_tokenize)
X['comments'] = X['comments'].apply(replace_slang_word, slang_word=indo_slang_word)
X['comments'] = X['comments'].apply(lemma_indo)
# remove stopword

# convert back to string
X['comments'] = X['comments'].apply(lambda x: ' '.join(x))
# X['comments'] = X['comments'].apply(stopword_removal)
print(X['label'].value_counts())
X['comments'] = X['comments'].astype(str)
true_label = X['label']
pos, neg, predictionLabel = process_and_save_data(X, prediction, './temp.csv')

print("\n")
from sklearn.metrics import classification_report
print(classification_report(true_label, predictionLabel))

# accuracy 76%

label
0.0    319
1.0     51
Name: count, dtype: int64


C:\Users\Aurel\AppData\Local\Temp\ipykernel_24520\3654328601.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comments'] = data['comments'].apply(clean_text)


label
0.0    305
1.0     48
Name: count, dtype: int64
Positive: 86
Negative: 267
Positive percentage: 24.362606232294617
Negative percentage: 75.63739376770539


              precision    recall  f1-score   support

         0.0       0.90      0.79      0.84       305
         1.0       0.24      0.44      0.31        48

    accuracy                           0.74       353
   macro avg       0.57      0.61      0.58       353
weighted avg       0.81      0.74      0.77       353



# Model Evaluation

* this one need research (pat)

# Analysis the data from primary data

## Modal Comprarison

# Conclusion